<a href="https://colab.research.google.com/github/mosesyhc/de300-wn2024-notes/blob/main/examples/ex-pipeline-news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Google drive for a permanent venv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Retrieving Java, Spark, and `findspark` in Python

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark
!pip install -q seaborn

In [ ]:
# spark setup
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [ ]:
# findspark helps locate the environment variables
import findspark
findspark.init()

# `agnews` Dataset

In [ ]:
!curl https://raw.githubusercontent.com/mosesyhc/de300-wn2024-notes/main/lab/dataset/agnews.csv -O

# Pipelining with PySpark MLlib

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline # pipeline to transform data


In [ ]:
spark = (SparkSession.builder
         .master("local[*]")
         .appName("AG news")
         .getOrCreate()
        )
sc = spark.sparkContext

In [ ]:
# load dataset
df = spark.read.csv("agnews.csv", inferSchema=True, header=True)

In [ ]:
df.show(20)

# Arrange columns

In [ ]:
from pyspark.sql.functions import concat_ws, col # to concatinate cols

# Renaming 'Class Index' col to 'label'
df = df.withColumnRenamed('Class Index', 'label')

# Add a new column 'text' by concatinating 'Title' and 'Description'
df = df.withColumn("text", concat_ws(" ", "Title", 'Description'))

# Remove old text columns
df = df.select('label', 'text')

# Shows top 10 rows
df.show(10)


# Tokenize

In [ ]:
from pyspark.ml.feature import RegexTokenizer # tokenizer

# convert sentences to list of words
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

# applies tokenizer to df
df = tokenizer.transform(df)

df.select(['label','text', 'words']).show(5)


# Stopwords

In [ ]:
from pyspark.ml.feature import StopWordsRemover

# remove stopwords
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered")

# adds a column 'filtered' to df without stopwords
df = stopwords_remover.transform(df)

df.select(['label','text', 'words', 'filtered']).show(5)

# Term frequency, Inverse document frequency

In [ ]:
from pyspark.ml.feature import HashingTF

# Calculate term frequency in each article
hashing_tf = HashingTF(inputCol="filtered",
                       outputCol="raw_features",
                       numFeatures=16384)

# adds raw tf features to df
featurized_data = hashing_tf.transform(df)

featurized_data.show(5)

In [ ]:
from pyspark.ml.feature import IDF
# inverse document frequency
idf = IDF(inputCol="raw_features", outputCol="features")

idf_vectorizer = idf.fit(featurized_data)

# converting text to vectors
rescaled_data = idf_vectorizer.transform(featurized_data)

rescaled_data.show(5)

In [ ]:
rescaled_data.select('raw_features').show(1, truncate=False)
rescaled_data.select('features').show(1, truncate=False)

# Training a multinomial logistic regression

In [ ]:
(train, test) = rescaled_data.randomSplit([0.75, 0.25])
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol='features',
                        labelCol='label',
                        family="multinomial",
                        regParam=0.3,
                        elasticNetParam=0,
                        maxIter=20)

lrModel = lr.fit(train)

# Prediction and evaluation

In [ ]:
# predict on test data
predictions = lrModel.transform(test)

In [ ]:
predictions.show(3)

In [ ]:
predictions.select("text", 'probability').show()

In [ ]:
predictions.select("text", 'probability', 'prediction', 'label').show()

In [ ]:
# accuracy flag
predictions = predictions.withColumn('correctFlag', (col('label') == col('prediction')))

In [ ]:
from pyspark.sql.functions import avg
from pyspark.sql.types import FloatType
# accuracy
predictions.select(avg(col('correctFlag').cast(FloatType())).alias('accuracy')).show()

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
# labels = ["World", "Sports", "Business","Science"]

# take only the predictions
preds_and_labels = predictions.select(['prediction','label']) \
                              .withColumn('label', col('label') \
                              .cast(FloatType())) \
                              .orderBy('prediction')

# generate confusion matrix counts
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

In [ ]:
# confusion matrix
metrics.confusionMatrix().toArray()

# Pipelining, from start to finish

In [ ]:
# load dataset
df = spark.read.csv("agnews.csv", inferSchema=True, header=True)

def arrangeColumns(df):
  # Renaming 'Class Index' col to 'label'
  df = df.withColumnRenamed('Class Index', 'label')

  # Add a new column 'text' by joining 'Title' and 'Description'
  df = df.withColumn("text", concat_ws(" ", "Title", 'Description'))

  # Select new text feature and labels
  df = df.select('label', 'text')
  return df

df = arrangeColumns(df)

# tokenizer
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

# stopwords
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered")

# term frequency
hashing_tf = HashingTF(inputCol="filtered",
                       outputCol="raw_features",
                       numFeatures=16384)

# Inverse Document Frequency
idf = IDF(inputCol="raw_features", outputCol="features")

# model
lr = LogisticRegression(featuresCol='features',
                        labelCol='label',
                        family="multinomial",
                        regParam=0.3,
                        elasticNetParam=0,
                        maxIter=20)


In [ ]:
# Put everything in pipeline
pipeline = Pipeline(stages=[tokenizer,
                            stopwords_remover,
                            hashing_tf,
                            idf,
                            lr])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df)

# transform add train
dataset = pipelineFit.transform(df)

In [ ]:
# accuracy
dataset = dataset.withColumn('correctFlag', (col('label') == col('prediction')))
dataset.select(avg(col('correctFlag').cast(FloatType())).alias('accuracy')).show()